In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_pickle("/kaggle/input/data0-25/data_norm0.25.pkl")
df2 = pd.read_pickle("/kaggle/input/data1-0/data_norm1.pkl")
df3 = pd.read_pickle("/kaggle/input/data0-5/data_norm0.5.pkl")

In [3]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])

In [4]:
df = df.reset_index() 

In [5]:
df = df.drop("index",  axis=1)

In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=127)

In [7]:
del df1
del df2
del df3

In [8]:
df

,OUTPUT,INPUT
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1242904446...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0770836309982...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0.10748772366243657, 0, 0, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0.06577925136276556, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1292574,"[0, 0, 0.23119837918566855, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292575,"[0, 0, 0, 0, 0, 0, 0, 0.07482285803921773, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292576,"[0, 0, 0.13130826532079368, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1292577,"[0.0866654566110813, 0.0684285664685669, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
df_test = df_test.reset_index() 
df_train = df_train.reset_index() 

In [10]:
df_train = df_train.drop("index",  axis=1)
df_test = df_test.drop("index",  axis=1)

In [13]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float)
        y = torch.tensor(self.data.iloc[idx, 1], dtype=torch.float)
        return x, y

 
datatest = CustomDataset(df_test) 
dataloader_test = DataLoader(datatest, batch_size=32, shuffle=False)

In [11]:
import torch
import torch.nn as nn


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(989, 49)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

In [12]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
model = CustomModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
threshold = 0.10

for epoch in range(20):
    for targets, inputs in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
            
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
            
            
        outputs = outputs.detach().numpy()
        targets = targets.detach().numpy()
            
        outputs = (outputs >= threshold).astype(int)
        targets = (targets >= threshold).astype(int)
            
        outputs = np.argmax(outputs, axis=1)
        targets = np.argmax(targets, axis=1)
            
        precision = np.mean(precision_score(targets, outputs, average=None, zero_division=1))
        recall = np.mean(recall_score(targets, outputs, average=None, zero_division=1))


    print("Epoch:", epoch + 1, "loss:", loss.item(), "recall:", recall, "precision:", precision)
    torch.save(model, '/kaggle/working/989_x3_49_epoch' + str(epoch + 1) + '.pth')
            
            





In [ ]:
model = torch.load("/kaggle/input/version3/pytorch/models/1/989_x2_256_49_epoch2.pth")
from IPython.display import clear_output
def clear_console():
    clear_output(wait=True)



import time
for n in range(0, 1000000, 1000):
    out = model.forward(torch.tensor(df["INPUT"][n])).tolist()
    rounded_out = [round(num, 3) for num in out]
    print("Выход сети:" + "      "+ "Датасет:")
    for i in range(49):
        if rounded_out[i] > 0:
            print(rounded_out[i], " " * (12-len(str(rounded_out[i]))+3),
                  round(df["OUTPUT"][n][i], 3))
        else:
            print(rounded_out[i], " " * 12, round(df["OUTPUT"][n][i], 3))
    time.sleep(6)
    clear_console()

Процесс самого обучения не записался из-за очередного бага вывода в kaggle, но тесты на тестовых данные преставлены в файле evaluate_model_v4.0.ipynb